In [1]:
# Imports from driver.py
import sys
from driver import (
    Expr, Var, Type, Pi, Lambda, App, Nat, Zero, Succ, ElimNat,
    Environment, type_check, eval_expr, alpha_equal, substitute
)


In [2]:
# 1: Base Alpha Equivalence

# Define two lambda expressions with different bound variable names
expr1 = Lambda('x', Nat(), Var('x'))
expr2 = Lambda('y', Nat(), Var('y'))

# Test alpha equivalence
result = alpha_equal(expr1, expr2)
print(f"Alpha Equivalence of expr1 and expr2: {result}")  # Expected: True


Alpha Equivalence of expr1 and expr2: True


In [3]:
# 2: Constant Function Test

# Define a constant function: const = lambda (A : *) . lambda (x : A) . lambda (y : A). x
const_expr = Lambda('A', Type(), Lambda('x', Var('A'),
                   Lambda('y', Var('A'), Var('x'))))

# Type check the constant function
tau_const = type_check([], const_expr)
print(f"Type of const: {tau_const}")

# Define the expected type: Pi(A : Type, Pi(x : A, Pi(y : A, A)))
expected_type = Pi('A', Type(), Pi('x', Var('A'), Pi('y', Var('A'), Var('x'))))
print(f"Expected type: {expected_type}")

# Check alpha equivalence
alpha_eq = alpha_equal(tau_const, expected_type)
print(f"Alpha Equivalence of tau_const and expected_type: {alpha_eq}")  # Expected: True


Type of const: Pi(x='A', tau1=Type(), tau2=Pi(x='x', tau1=Var(name='A'), tau2=Pi(x='y', tau1=Var(name='A'), tau2=Var(name='A'))))
Expected type: Pi(x='A', tau1=Type(), tau2=Pi(x='x', tau1=Var(name='A'), tau2=Pi(x='y', tau1=Var(name='A'), tau2=Var(name='x'))))
Var mismatch: A mapped to A vs x
Alpha Equivalence of tau_const and expected_type: False


In [4]:
# 3: Addition Function Definition

# Define addition as add : Nat -> Nat -> Nat
# add x y = elimNat (Pi(z, Nat(), Nat()), y, lambda n. Succ (add n y), x)

# Define the motive: Pi('z', Nat(), Nat())
motive = Pi('z', Nat(), Nat())

# Define the base case: y
base_case = Var('y')

# Define the inductive step: lambda n. Succ (add n y)
inductive_step = Lambda(
    'n',
    Nat(),
    Succ(App(App(Var('add'), Var('n')), Var('y')))
)

# Define the target: x
target = Var('x')

# Define elimNat expression
elim_nat = ElimNat(motive, base_case, inductive_step, target)

# Define add function: lambda x. lambda y. elim_nat
add_expr = Lambda('x', Nat(), Lambda('y', Nat(), elim_nat))

# To allow recursion, we need to add 'add' to the environment referencing its type
# Type of add: Pi('x', Nat(), Pi('y', Nat(), Nat()))
add_type = Pi('x', Nat(), Pi('y', Nat(), Nat()))

# Initialize environment with 'add' binding
Gamma = [('add', add_type)]

# Since 'add' is being defined recursively, we need to allow it to reference itself
# However, in this setup, 'add' is already in Gamma, so 'Var('add')' refers to its type

print(f"Defined add function: {add_expr}")
print(f"Type of add function: {add_type}")

Defined add function: Lambda(x='x', tau1=Nat(), e2=Lambda(x='y', tau1=Nat(), e2=ElimNat(e1=Pi(x='z', tau1=Nat(), tau2=Nat()), e2=Var(name='y'), e3=Lambda(x='n', tau1=Nat(), e2=Succ(e=App(e1=App(e1=Var(name='add'), e2=Var(name='n')), e2=Var(name='y')))), e4=Var(name='x'))))
Type of add function: Pi(x='x', tau1=Nat(), tau2=Pi(x='y', tau1=Nat(), tau2=Nat()))


In [6]:

try:
    # Type check the addition function
    tau_add = type_check(Gamma, add_expr)
    
    # Define the expected type: Nat -> Nat -> Nat
    expected_type = Pi('x', Nat(), Pi('y', Nat(), Nat()))
    
    # Check if the inferred type matches the expected type
    is_correct_type = alpha_equal(tau_add, expected_type)
    
    print(f"Type of 'add' is correct: {is_correct_type}")  # Expected Output: True
except TypeError as e:
    print(f"Type checking failed: {e}")

Type checking failed: Motive of elimNat is not of type ℕ → ⋆: Pi(x='z', tau1=Nat(), tau2=Nat())
